# Day 2 - Lab 1: AI-Generated System Design & Database Seeding

**Objective:** Use the PRD artifact from Day 1 to generate a detailed SQL database schema, create realistic seed data, and then use those outputs to create and seed a live, local database file.

**Estimated Time:** 150 minutes

**Introduction:**
Welcome to Day 2! Today, we transition from *what* we're building to *how* we'll build it. In this lab, you will act as the lead architect for the Onboarding Tool. Your task is to use the PRD to define the data structure of the application and create a tangible database artifact that will be used for the rest of the course.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the `day1_prd.md` artifact from Day 1. This document is the primary source of truth for our project and provides the necessary context for the LLM to make intelligent design suggestions.

**Model Selection:**
Feel free to experiment with different models by changing the `model_name` in `setup_llm_client()`. Models with strong reasoning capabilities, like `gpt-4o`, `o3`, or `gemini-2.5-pro`, are excellent choices for design tasks.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the PRD from the `artifacts` directory.
- `save_artifact()`: To save the generated SQL schema and seed data.
- `clean_llm_output()`: To remove markdown fences from the generated SQL code.

In [1]:
import sys
import os
import sqlite3

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output,prompt_enhancer

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

# Load the PRD from Day 1
prd_content = load_artifact("artifacts/waffle_PRD_output.md")

if not prd_content:
    print("Warning: Could not load waffle_PRD_output.md. Lab may not function correctly.")

2025-10-02 12:50:26,334 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4o latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating the SQL Schema

**Task:** Use the PRD to generate a normalized SQL schema for the application.

**Instructions:**
1.  Create a prompt that instructs the LLM to act as a Database Administrator (DBA).
2.  Provide the `prd_content` as context.
3.  Ask the LLM to design a normalized SQL schema with at least two tables (e.g., `users` and `onboarding_tasks`).
4.  The output should be the raw `CREATE TABLE` statements.
5.  Save the generated SQL to `artifacts/schema.sql`.

In [2]:
# TODO: Write a prompt to generate the SQL schema from the PRD.
schema_prompt = f"""
You are a Senior Database Architect. Design a comprehensive, production-ready SQL database schema based on the Product Requirements Document (PRD) provided below.

**Requirements:**
- Create not more than 2 tables (e.g., users and user_roles)
- Include all necessary tables to fully support the application's features and workflows
- Define appropriate columns with proper data types for each table
- Establish primary keys for all tables
- Define foreign keys and relationships between tables where applicable
- Follow database normalization best practices (at least 3NF)
- Use standard SQL naming conventions (lowercase with underscores)
- Include appropriate indexes for performance optimization
- Add constraints (NOT NULL, UNIQUE, CHECK) where logically necessary
- Consider data integrity, scalability, and query performance
- Use AUTO_INCREMENT for primary keys; instead
- Use SQL Lite compatible syntax only

**Output Format:**
Provide ONLY the raw SQL CREATE TABLE statements without:
- Markdown code blocks or formatting
- Explanatory text or comments
- Additional documentation

Each CREATE TABLE statement should be complete and executable.

**PRD:**
{prd_content}
"""

#enhanced_schema_prompt = prompt_enhancer(schema_prompt)
print("--- Generating SQL Schema ---")
if prd_content:
    generated_schema = get_completion(schema_prompt, client, model_name, api_provider)
    print(generated_schema)
    
    # Clean up the generated schema using our helper function
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    print(cleaned_schema)
    
    # Save the cleaned schema
    save_artifact(cleaned_schema, 'artifacts/waffle_schema.sql',overwrite=True)
else:
    print("Skipping schema generation because PRD is missing.")
    cleaned_schema = ""

--- Generating SQL Schema ---
create table users (
    user_id integer primary key,
    first_name text not null,
    last_name text not null,
    email text not null unique,
    hire_date text not null,
    role_id integer not null,
    bio text,
    check (email like '%@%')
);

create table user_roles (
    role_id integer primary key,
    role_name text not null unique
);

create index idx_users_role_id on users(role_id);

alter table users add foreign key (role_id) references user_roles(role_id);
create table users (
    user_id integer primary key,
    first_name text not null,
    last_name text not null,
    email text not null unique,
    hire_date text not null,
    role_id integer not null,
    bio text,
    check (email like '%@%')
);

create table user_roles (
    role_id integer primary key,
    role_name text not null unique
);

create index idx_users_role_id on users(role_id);

alter table users add foreign key (role_id) references user_roles(role_id);


### Challenge 2 (Intermediate): Generating Realistic Seed Data

**Task:** Prompt the LLM to generate realistic seed data that conforms to the schema you just created.

**Instructions:**
1.  Create a new prompt.
2.  Provide both the `prd_content` and the `cleaned_schema` as context.
3.  Instruct the LLM to generate 5-10 realistic `INSERT` statements for your tables.
4.  The data should be relevant to a new hire onboarding tool (e.g., sample user names and task titles like "Complete HR Paperwork").
5.  Save the generated `INSERT` statements to `artifacts/seed_data.sql`.

In [3]:
# TODO: Write a prompt to generate realistic seed data.
seed_data_prompt = f"""
You are a Database Analyst specializing in test data generation. Your task is to create realistic seed data that is STRICTLY COMPLIANT with the provided database schema.

**Critical Requirements:**
- Generate exactly 10 INSERT statements for EACH table defined in the schema
- ALL data must conform to the exact data types specified in the schema (VARCHAR lengths, INTEGER ranges, DATE formats, etc.)
- STRICTLY respect all constraints defined in the schema:
  * PRIMARY KEY constraints (ensure uniqueness)
  * FOREIGN KEY constraints (ensure referential integrity - all FK values must reference existing PK values)
  * NOT NULL constraints (provide values for ALL NOT NULL columns)
  * UNIQUE constraints (ensure no duplicates)
  * CHECK constraints (satisfy all validation rules)
  * DEFAULT values (you may use them or override with explicit values)
- Maintain proper insert order: parent tables BEFORE child tables to satisfy FK dependencies
- Use realistic, contextually appropriate data for a new hire onboarding tool

**Data Guidelines:**
- User names: realistic full names (e.g., "Sarah Johnson", "Michael Chen")
- Task titles: relevant onboarding activities (e.g., "Complete I-9 Form", "Setup Workstation", "Attend Security Training")
- Dates: use logical sequences (hire dates before task due dates, etc.)
- Email addresses: follow standard format (firstname.lastname@company.com)
- Status values: match any ENUM or CHECK constraint definitions exactly
- IDs: start from 1 and increment sequentially

**Output Format:**
Provide ONLY executable SQL INSERT statements:
- No markdown formatting or code blocks
- No explanatory comments or documentation
- One INSERT statement per line or proper multi-row INSERT syntax
- Statements must be ready to execute immediately after CREATE TABLE statements

**Context:**
PRD: {prd_content}
Schema: {cleaned_schema}

**Validation Checklist (verify before output):**
□ All FK values reference existing PK values
□ All NOT NULL columns have values
□ All data types match schema exactly
□ All string lengths within VARCHAR limits
□ Proper insert order maintained
□ All constraints satisfied
"""
#enhanced_seed_data_prompt = prompt_enhancer(seed_data_prompt)

print("--- Generating Seed Data ---")
if prd_content and cleaned_schema:
    generated_seed_data = get_completion(seed_data_prompt, client, model_name, api_provider)
    
    # Clean up the generated seed data
    cleaned_seed_data = clean_llm_output(generated_seed_data, language='sql')
    print(cleaned_seed_data)
    
    # Save the cleaned seed data
    save_artifact(cleaned_seed_data, 'artifacts/waffle_seed_data.sql',overwrite=True)
else:
    print("Skipping seed data generation because PRD or schema is missing.")

--- Generating Seed Data ---
insert into user_roles (role_id, role_name) values (1, 'Software Engineer');
insert into user_roles (role_id, role_name) values (2, 'HR Manager');
insert into user_roles (role_id, role_name) values (3, 'Product Manager');
insert into user_roles (role_id, role_name) values (4, 'Design Lead');
insert into user_roles (role_id, role_name) values (5, 'Sales Executive');
insert into user_roles (role_id, role_name) values (6, 'Customer Support');
insert into user_roles (role_id, role_name) values (7, 'Marketing Specialist');
insert into user_roles (role_id, role_name) values (8, 'DevOps Engineer');
insert into user_roles (role_id, role_name) values (9, 'Data Analyst');
insert into user_roles (role_id, role_name) values (10, 'Finance Officer');

insert into users (user_id, first_name, last_name, email, hire_date, role_id, bio) values (1, 'Sarah', 'Johnson', 'sarah.johnson@company.com', '2023-01-15', 1, 'Experienced software engineer with a passion for developing in

### Challenge 3 (Advanced): Creating and Seeding a Live Database

**Task:** This is a critical technical step. You will write a Python script to execute the generated SQL files, creating a live `onboarding.db` file that your application will use.

**Instructions:**
1.  Complete the `create_database` function below.
2.  The function should first connect to (and thus create) a SQLite database file named `artifacts/onboarding.db`.
3.  It should then open and execute the `schema.sql` file to create the tables.
4.  Finally, it should open and execute the `seed_data.sql` file to populate the tables.
5.  Use a `try...finally` block to ensure the database connection is always closed, even if an error occurs.

> **Hint:** The `try...finally` block is a crucial Python pattern. The code in the `finally` block will run whether the `try` block succeeds or fails, making it the perfect place to ensure resources like database connections are always closed.

**Expected Quality:** A physical `onboarding.db` file in your `artifacts` folder. This is a tangible asset that proves your design is valid and provides a concrete foundation for backend development.

In [4]:
import re
def tables_to_drop(schema_path):
    """Extracts all table names from CREATE TABLE statements in the schema file."""
    if not os.path.exists(schema_path):
        print(f"Error: Schema file not found at {schema_path}")
        return []
    with open(schema_path, 'r', encoding='utf-8') as f:
        sql = f.read()
    # Regex to match CREATE TABLE [IF NOT EXISTS] table_name
    pattern = r"CREATE TABLE(?: IF NOT EXISTS)?\s+([a-zA-Z0-9_]+)"
    tables = re.findall(pattern, sql, re.IGNORECASE)
    return tables

In [6]:
import os
def create_database(db_path, schema_path, seed_path):
    """Creates and seeds a SQLite database from SQL files."""
    if not os.path.exists(schema_path):
        print(f"Error: Schema file not found at {schema_path}")
        return
    
    conn = None
    try:
        # Connect to the SQLite database. This will create the file if it doesn't exist.
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print(f"Successfully connected to database at {db_path}")

        # --- Drop ALL existing tables in the database ---
        print("Checking for existing tables in database...")
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
        existing_tables = cursor.fetchall()
        
        if existing_tables:
            print(f"Found {len(existing_tables)} existing tables: {[table[0] for table in existing_tables]}")
            # Disable foreign key constraints temporarily to allow dropping tables
            cursor.execute("PRAGMA foreign_keys = OFF;")
            
            for table in existing_tables:
                table_name = table[0]
                cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
                print(f"  Dropped table: {table_name}")
            
            # Re-enable foreign key constraints
            cursor.execute("PRAGMA foreign_keys = ON;")
            print("All existing tables dropped successfully.")
        else:
            print("No existing tables found in database.")

        # --- Additional safety: Drop tables from schema.sql if any remain ---
        tables_from_schema = tables_to_drop(schema_path)
        for table in tables_from_schema:
            cursor.execute(f"DROP TABLE IF EXISTS {table};")
        print("Schema-based table cleanup completed.")

        # Read the content of the schema file using load_artifact.
        schema_sql = load_artifact(schema_path)
        
        # Replace TIMESTAMP WITH TIME ZONE with TIMESTAMP for SQLite compatibility
        if schema_sql:
            schema_sql = schema_sql.replace("TIMESTAMP WITH TIME ZONE", "TIMESTAMP")
            cursor.executescript(schema_sql)
            print("Tables created successfully.")

        # Check if the seed data file exists. If it does, load and execute it.
        if os.path.exists(seed_path):
            seed_sql = load_artifact(seed_path)
            if seed_sql:
                cursor.executescript(seed_sql)
                print("Seed data inserted successfully.")

        # Commit the changes to the database.
        conn.commit()
        print("Database changes committed.")    
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        # Ensure the connection is closed if it was opened.
        if conn:
            conn.close()
            print("Database connection closed.")

def query_table(db_path, query, params=None):
    if not os.path.exists(db_path):
        print(f"Error: Database file not found at {db_path}")
        return None
    conn = None
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        # Execute the query with or without parameters
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
        # Fetch all results
        results = cursor.fetchall()
        # Get column names for better readability (optional)
        column_names = [description[0] for description in cursor.description]
        print(f"Query executed successfully. Found {len(results)} rows.")
        print(f"Columns: {', '.join(column_names)}")
        return results
    except sqlite3.Error as e:
        print(f"Database query error: {e}")
        return None
    finally:
        # Ensure the connection is closed if it was opened
        if conn:
            conn.close()
try:
    project_root
except NameError:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

db_file = os.path.join(project_root, "artifacts", "waffle_tech_suite.db")
schema_file = os.path.join(project_root, "artifacts", "waffle_schema.sql")
seed_file = os.path.join(project_root, "artifacts", "waffle_seed_data.sql")

# Execute the function
create_database(db_file, schema_file, seed_file)




Successfully connected to database at c:\Users\labadmin\Desktop\Repository\AG-AISOFTDEV\artifacts\waffle_tech_suite.db
Checking for existing tables in database...
Found 2 existing tables: ['users', 'user_roles']
  Dropped table: users
  Dropped table: user_roles
All existing tables dropped successfully.
Schema-based table cleanup completed.
Tables created successfully.
Seed data inserted successfully.
Database changes committed.
Database connection closed.


In [7]:
# Verify the database was created and populated correctly
print("=== Database Verification ===")

# Query user_roles table
print("\n--- User Roles ---")
roles_results = query_table(db_file, "SELECT * FROM user_roles ORDER BY role_id")
if roles_results:
    for row in roles_results:
        print(f"Role ID: {row[0]}, Role Name: {row[1]}")

# Query users table
print("\n--- Users ---")
users_results = query_table(db_file, "SELECT * FROM users ORDER BY user_id")
if users_results:
    for row in users_results:
        print(f"User ID: {row[0]}, Name: {row[1]} {row[2]}, Email: {row[3]}, Role ID: {row[5]}")

# Query with JOIN to show user roles
print("\n--- Users with Role Names ---")
join_results = query_table(db_file, """
    SELECT u.user_id, u.first_name, u.last_name, u.email, ur.role_name 
    FROM users u 
    JOIN user_roles ur ON u.role_id = ur.role_id 
    ORDER BY u.user_id
""")
if join_results:
    for row in join_results:
        print(f"User: {row[1]} {row[2]} ({row[3]}) - Role: {row[4]}")

=== Database Verification ===

--- User Roles ---
Query executed successfully. Found 10 rows.
Columns: role_id, role_name
Role ID: 1, Role Name: Software Engineer
Role ID: 2, Role Name: HR Manager
Role ID: 3, Role Name: Product Manager
Role ID: 4, Role Name: Design Lead
Role ID: 5, Role Name: Sales Executive
Role ID: 6, Role Name: Customer Support
Role ID: 7, Role Name: Marketing Specialist
Role ID: 8, Role Name: DevOps Engineer
Role ID: 9, Role Name: Data Analyst
Role ID: 10, Role Name: Finance Officer

--- Users ---
Query executed successfully. Found 10 rows.
Columns: user_id, first_name, last_name, email, hire_date, role_id, bio
User ID: 1, Name: Sarah Johnson, Email: sarah.johnson@company.com, Role ID: 1
User ID: 2, Name: Michael Chen, Email: michael.chen@company.com, Role ID: 2
User ID: 3, Name: Emily Smith, Email: emily.smith@company.com, Role ID: 3
User ID: 4, Name: James Brown, Email: james.brown@company.com, Role ID: 4
User ID: 5, Name: Linda Davis, Email: linda.davis@company.

In [8]:
# Final status check
import os

print("=== Lab Completion Status ===")
print(f"✅ Database file created: {os.path.exists(db_file)}")
print(f"✅ Schema file exists: {os.path.exists(schema_file)}")
print(f"✅ Seed data file exists: {os.path.exists(seed_file)}")
print(f"✅ Database contains {len(users_results)} users")
print(f"✅ Database contains {len(roles_results)} user roles")
print(f"\n🎉 Lab completed successfully!")
print(f"Database location: {db_file}")

=== Lab Completion Status ===
✅ Database file created: True
✅ Schema file exists: True
✅ Seed data file exists: True
✅ Database contains 10 users
✅ Database contains 10 user roles

🎉 Lab completed successfully!
Database location: c:\Users\labadmin\Desktop\Repository\AG-AISOFTDEV\artifacts\waffle_tech_suite.db


## Lab Conclusion

Excellent work! You have now moved from abstract requirements to a concrete, physical database artifact. You've used an LLM to design a schema, generate realistic test data, and then used a Python script to bring that database to life. This `onboarding.db` file is the foundation upon which we will build our API in Day 3.

> **Key Takeaway:** The ability to generate structured data definitions (like a SQL schema) from unstructured text (like a PRD) is a core skill in AI-assisted development. It automates a critical and often time-consuming design step.